# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df_weather = pd.read_csv("../output_data/Cities.csv")
df_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bluff,-46.6000,168.3333,48.99,75,32,3.00,NZ,1620781739
1,tiksi,71.6872,128.8694,22.77,89,51,3.27,RU,1620781740
2,kukushtan,57.6464,56.4952,48.20,81,0,3.51,RU,1620781740
3,ewa beach,-8.6235,122.5073,78.44,61,51,3.96,ID,1620781741
4,christchurch,-43.5333,172.6333,52.00,87,75,16.11,NZ,1620781702


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# distinguish humidity and locations for lat and long
locations = df_weather[['Lat', 'Lng']]
humidity = df_weather['Humidity'].astype(float)
max_humidity = df_weather['Humidity'].max()

In [5]:
# add heatmap layer to map
fig = gmaps.figure(center=(30.0, -5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=2)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# my ideal weather conditions are:
# temps between 70-85 degrees
# cloudiness <=10%
# wind speed <= 5 mph


df_vacation = df_weather.loc[(df_weather['Max Temp'] >= 70) & (df_weather['Max Temp'] <= 85) & (df_weather['Cloudiness'] <=10) & (df_weather['Wind Speed'] <=5)]
df_vacation

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
65,namibe,-15.1961,12.1522,73.69,61,0,4.21,AO,1620781769
115,mecca,21.4267,39.8261,81.64,46,0,3.29,SA,1620781795
142,atambua,-9.1061,124.8925,79.90,60,2,2.95,ID,1620781809
334,bushehr,28.9684,50.8385,83.75,44,0,3.29,IR,1620781639
377,dubai,25.2582,55.3047,82.40,69,5,4.61,AE,1620781733
491,inongo,-1.9500,18.2667,72.28,88,0,3.67,CD,1620781989


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# create new dataframe and add new column called "Hotel Name" 
df_hotel = df_vacation [['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country']]
df_hotel['Hotel Name'] = ""
df_hotel

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
65,namibe,-15.1961,12.1522,73.69,61,0,4.21,AO,
115,mecca,21.4267,39.8261,81.64,46,0,3.29,SA,
142,atambua,-9.1061,124.8925,79.90,60,2,2.95,ID,
334,bushehr,28.9684,50.8385,83.75,44,0,3.29,IR,
377,dubai,25.2582,55.3047,82.40,69,5,4.61,AE,
491,inongo,-1.9500,18.2667,72.28,88,0,3.67,CD,


In [8]:
# set base url and parameters to search for hotels with 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type": "hotel",
          "radius": 5000,
          "keyword": "hotel",
          "key": g_key}

In [9]:
# use iterrows to iterate through dataframe
for index, row in df_hotel.iterrows():
    
    # get lat, lng and city from dataframe
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # store the first Hotel result into the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        
        df_hotel.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("--------------------")

Retrieving Results for Index 65: namibe.
Closest hotel in namibe is Hotel Chik Chik Namibe.
--------------------
Retrieving Results for Index 115: mecca.
Closest hotel in mecca is Pullman Zamzam Makkah.
--------------------
Retrieving Results for Index 142: atambua.
Closest hotel in atambua is Permata Hotel.
--------------------
Retrieving Results for Index 334: bushehr.
Closest hotel in bushehr is Sky 2 hotel.
--------------------
Retrieving Results for Index 377: dubai.
Closest hotel in dubai is Dubai International Hotel.
--------------------
Retrieving Results for Index 491: inongo.
Closest hotel in inongo is Hotel Du Lac.
--------------------


In [10]:
df_hotel

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
65,namibe,-15.1961,12.1522,73.69,61,0,4.21,AO,Hotel Chik Chik Namibe
115,mecca,21.4267,39.8261,81.64,46,0,3.29,SA,Pullman Zamzam Makkah
142,atambua,-9.1061,124.8925,79.90,60,2,2.95,ID,Permata Hotel
334,bushehr,28.9684,50.8385,83.75,44,0,3.29,IR,Sky 2 hotel
377,dubai,25.2582,55.3047,82.40,69,5,4.61,AE,Dubai International Hotel
491,inongo,-1.9500,18.2667,72.28,88,0,3.67,CD,Hotel Du Lac


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df_hotel.iterrows()]
locations = df_hotel[["Lat", "Lng"]]

In [12]:
# add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))